In [25]:
import numpy as np
import pandas as pd
import re
import html
import ast

# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords

# nltk.download('stopwords')
# nltk.download('punkt')

## Read data

In [2]:
data = pd.read_csv('concatenatedAll.csv')

In [3]:
data

,text,id,author_id,TentativeClass,Class
0,@pltxknower @Estries1 @anarcho_parm Oh no for ...,1432855547187970000,2822594617,1,1.0
1,A Guide on How to Get Abortion Pills | Plan C ...,1432855498873830000,153642560,1,1.0
2,"WTF!!! Abbott and male Republicans: MINE, AN...",1432855481127730000,3449791700,1,1.0
3,@pqben42 No that should be left up to the stat...,1432855475935140000,4012849515,1,2.0
4,i’m glad to see folks using their platforms to...,1432855417663720000,1134540000000000000,1,2.0
...,...,...,...,...,...
36691,Well this says it all. Sad sad sad. #ProLife #...,1434668855201250000,20290217,0,1.0
36692,@RammJaeger that is not prolife.... it is FORC...,1434668348441180000,2439831350,0,1.0
36693,"Last week, 200 Texas kids lay in ICU beds sick...",1434668250244290000,23619366,0,2.0
36694,@joshcecc @_pish @HowlingAtMoons @gael_lover @...,1434668245672460000,821571000000000000,0,0.0


In [4]:
#important_hashtags = ['probirth','prolife','proabortion','antiabortion','prochoice']

In [5]:
def process_tweet(text):
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = html.unescape(text)
    text = re.sub('[^A-Za-z ]+', '', text)
    text = text.strip().lower()
    
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(text)
    word_tokens = [w for w in word_tokens if (not w in stop_words) and (len(w)>1) and (len(w)<25)]
    text = ' '.join(word_tokens)
    
    return text

In [6]:
data['text'] = data['text'].apply(process_tweet)

In [7]:
data

,text,id,author_id,TentativeClass,Class
0,oh sure everything ive heard carrying child te...,1432855547187970000,2822594617,1,1.0
1,guide get abortion pills plan,1432855498873830000,153642560,1,1.0
2,wtf abbott male republicans mine every womans ...,1432855481127730000,3449791700,1,1.0
3,left states like abortion gay rights,1432855475935140000,4012849515,1,2.0
4,im glad see folks using platforms talk texas a...,1432855417663720000,1134540000000000000,1,2.0
...,...,...,...,...,...
36691,well says sad sad sad prolife satanic temple j...,1434668855201250000,20290217,0,1.0
36692,prolife forced birth fetuses babies matter man...,1434668348441180000,2439831350,0,1.0
36693,last week texas kids lay icu beds sick wcovid ...,1434668250244290000,23619366,0,2.0
36694,need take responsibility vast majority unplann...,1434668245672460000,821571000000000000,0,0.0


In [8]:
p_data = data[data.text.str.len() > 0]

In [9]:
p_data[['text','id','author_id','Class']].to_csv('PreprocessedAll.csv',index=False)

In [159]:
#temptweets.drop(['correct_author_id','Class'],axis=1).to_csv('preprocessed_tweets_correct.csv',index=False)

In [148]:
temptweets=pd.read_csv('PreprocessedAll.csv')
tempusers=pd.read_csv('ConcatenatedUsers_m3_eth.csv')
tempusers.rename(columns={'id':'author_id'},inplace=True)
tempusers.drop_duplicates('author_id',inplace=True)
tempusers['author_id']=tempusers['author_id'].astype(str)
temptweets['author_id']=temptweets['author_id'].astype(str)
tempusers['NewClass'] = 1

In [154]:
f = open('classDict.txt','r')
classDict = ast.literal_eval(f.readlines()[0])

In [128]:
len(set(temptweets.author_id)-set(tempusers.author_id))

1

In [110]:
m =pd.merge(temptweets,tempusers,on='author_id')

In [109]:
m['opinion'] = m.author_id.map(classDict)
m.opinion.value_counts()

1.0    342
0.0    206
Name: opinion, dtype: int64

In [152]:
temptweets

,text,id,author_id,Class,correct_author_id
0,oh sure everything ive heard carrying child te...,1432855547187970000,2822594617,1.0,NaN
1,guide get abortion pills plan,1432855498873830000,153642560,1.0,NaN
2,wtf abbott male republicans mine every womans ...,1432855481127730000,3449791700,1.0,NaN
3,left states like abortion gay rights,1432855475935140000,4012849515,2.0,NaN
4,im glad see folks using platforms talk texas a...,1432855417663720000,1134540000000000000,2.0,NaN
...,...,...,...,...,...
36156,well says sad sad sad prolife satanic temple j...,1434668855201250000,20290217,1.0,NaN
36157,prolife forced birth fetuses babies matter man...,1434668348441180000,2439831350,1.0,NaN
36158,last week texas kids lay icu beds sick wcovid ...,1434668250244290000,23619366,2.0,NaN
36159,need take responsibility vast majority unplann...,1434668245672460000,821571162736050176,0.0,821571162736050176


In [126]:
correctMapping={}

for badId in list(diff):
    temp=badId[:6]
    try:
        userrow=tempusers[tempusers.author_id.str.startswith(temp)].iloc[0]
        correctMapping[badId]=userrow.author_id
    except:
        pass

In [145]:
str(temptweets.iloc[0].correct_author_id)

'nan'

In [149]:
temptweets['correct_author_id']=temptweets.author_id.map(correctMapping)

In [151]:
for i,row in temptweets.iterrows():
    if str(row['correct_author_id'])!='nan':
        temptweets.at[i,'author_id']=row['correct_author_id']

In [160]:
tempUsers = pd.read_csv('concatenated_users_labelled.csv')

In [177]:
def isinTexas(x):
    x=str(x)
    x=x.lower()
    
    tx=['texas','tx','dallas','houston','san antonio','austin','fort worth','el paso']
    for i in tx:
        if i in x:
            return True
    return False

In [178]:
tempUsers['texas'] = tempUsers['location'].apply(isinTexas)

In [182]:
tempUsers[tempUsers.texas==1]

,description,created_at,profile_image_url,name,protected,verified,url,username,author_id,location,entities,pinned_tweet_id,gender,age,org,race,opinion,texas
1,"📍 Dallas, TX I show you where the adventure is...",2010-06-09T02:39:11.000Z,https://pbs.twimg.com/profile_images/143451186...,Dallas Latina Foodie,False,False,https://t.co/IfrZpDpvPs,allycelebrates,153642560,"Dallas, TX","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",1.450854e+18,female,19-29,non-org,white,1.0,1
15,"Native Texan, y'all! Mom, #Spurs fan, lover of...",2007-04-10T20:44:57.000Z,https://pbs.twimg.com/profile_images/144667817...,Sarah Loyd (she/her),False,False,https://t.co/ZfuyXn44L7,sarahloyd,4075241,"San Antonio, TX","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN,female,19-29,non-org,white,1.0,1
27,"News10: On Your Side. Bringing you weather, br...",2008-01-20T20:47:27.000Z,https://pbs.twimg.com/profile_images/132962502...,KWTX News 10,False,True,https://t.co/iBrvJA4tHt,kwtx,12469012,"Waco, Temple, Killeen, TX","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN,female,<=18,is-org,white,0.0,1
33,Wife/ Mom/ Independent Politically/News and Po...,2010-09-17T12:07:12.000Z,https://pbs.twimg.com/profile_images/242734299...,LoriAnn,False,False,NaN,Vogt1,191806576,"Katy, Texas",NaN,NaN,female,>=40,non-org,white,1.0,1
38,Good Texas Lawyer - from tickets to million do...,2012-07-01T03:46:14.000Z,https://pbs.twimg.com/profile_images/347720403...,winocour law,False,False,https://t.co/dX1oK9gWoB,goodtexaslawyer,623460684,"Dallas, TX","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN,male,>=40,is-org,black,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,https://t.co/K2ZJ4XsS6W\n\nGraphic Designer. P...,2013-05-09T14:14:44.000Z,https://pbs.twimg.com/profile_images/137600087...,French Burnt Peanuts,False,False,NaN,bishopsring_,1415654443,Texas,"{'description': {'urls': [{'start': 0, 'end': ...",1.448107e+18,male,<=18,non-org,white,1.0,1
19157,🇬🇹 • 26,2017-11-29T18:24:10.000Z,https://pbs.twimg.com/profile_images/117091049...,Viv,False,False,NaN,naiveviv,935937446272995328,"Dallas, TX",NaN,1.007660e+18,female,<=18,non-org,white,0.0,1
19179,"educator, thinker, planner, do gooder and rock...",2009-01-13T20:37:31.000Z,https://pbs.twimg.com/profile_images/134830953...,Julie HarrisLawrence,False,False,NaN,Jharrisl,18952264,Austin,NaN,NaN,female,>=40,non-org,white,1.0,1
19180,"47, End Stage Kidney Failure, lifelong dialysi...",2009-03-24T14:51:18.000Z,https://pbs.twimg.com/profile_images/125780624...,♥☆ ᒚᗩ᙭ ᒚᗩᘻᘿS ☆♥,False,False,NaN,lucki_jacki,26250667,"Austin, TX","{'description': {'hashtags': [{'start': 83, 'e...",NaN,female,<=18,non-org,white,1.0,1


In [183]:
tempUsers.texas.value_counts()

0    17966
1     1248
Name: texas, dtype: int64

In [184]:
tempUsers.to_csv('withtexas.csv',index=False)